In [1]:
import sys

from SALib.analyze import delta
from SALib.util import read_param_file
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("summaries_round_3.csv", sep=';')

df['ElecRate'] = 0.42
df["Pop2030"] = 15672000

df['ElecRate'] = np.where(df['Country'] == 'na', 0.54, df['ElecRate'])
df['ElecRate'] = np.where(df['Country'] == 'mw', 0.18, df['ElecRate'])

df['Pop2030'] = np.where(df['Country'] == 'na', 3010873, df['Pop2030'] )
df['Pop2030'] = np.where(df['Country'] == 'mw', 19130000, df['Pop2030'])

cols = ['EDemand',
 'GridGenCost',
 'PVCost',
 'GridCapInvCost',
 'DiscountRate',
 'LVCost',
 'MVCost',
 'GridLosses',
 'Core',
 'Buffer',
 'Admin',
 'Method',
 'Res',
 'ElecRate',
 'NatPopDens',
 'Pop2030',
 'PopGrid',
 'PopMG',
 'PopSA',
 'AddedCapGrid',
 'AddedCapSA',
 'AddedCapMG',
 'GridInv',
 'SAInv',
 'MGInv',
 'TotInv',
 'LCOEGrid',
 'LCOESA',
 'LCOEMGWind',
 'LCOEMGPV',
 'LCOEMGHydro',
 'LCOE',
 'Country']

df = df[cols]

df["RatioGridPop"] = 100*(df["PopGrid"]/df['Pop2030'])
df["RatioMGPop"] = 100*(df["PopMG"]/df['Pop2030'])
df["RatioSAPop"] = 100*(df["PopSA"]/df['Pop2030'])

df["RatioGridInv"] = 100*(df["GridInv"]/df['TotInv'])
df["RatioSAInv"] = 100*(df["SAInv"]/df['TotInv'])
df["RatioMGInv"] = 100*(df["MGInv"]/df['TotInv'])

df_shuffled = df.sample(frac=1)

FileNotFoundError: [Errno 2] No such file or directory: 'summaries_round_3.csv'

In [ ]:
dfB = df_shuffled.loc[df['Country'] == 'bj']
dfN = df_shuffled.loc[df['Country'] == 'na']
dfM = df_shuffled.loc[df['Country'] == 'mw']
dfB['id'] = np.arange(len(dfB))
dfN['id'] = np.arange(len(dfN))
dfM['id'] = np.arange(len(dfM))

In [ ]:
problem_merged = read_param_file('params_merged.txt')
problem_bj =read_param_file('params_ben.txt')
problem_na = read_param_file('params_nam.txt')
problem_mw = read_param_file('params_mal.txt')

## Merged

In [ ]:
param_values=df_shuffled.iloc[:, : 16]
np.savetxt(r'model_input_merged.txt', param_values.values)
X_merged = np.loadtxt(r"model_input_merged.txt")

In [ ]:
out = df_shuffled["RatioMGPop"]
np.savetxt(r'model_output_merged.txt', out.values)
Y_merged = np.loadtxt(r"model_output_merged.txt")

In [ ]:
S_merged = delta.analyze(problem_merged, X_merged, Y_merged, num_resamples=10, conf_level=0.95, print_to_console=True)

## Benin

In [ ]:
param_values=dfB.iloc[:, : 14]
np.savetxt(r'model_input_ben.txt', param_values.values)
X_bj = np.loadtxt(r"model_input_ben.txt")

In [ ]:
out = dfB["RatioMGPop"]
np.savetxt(r'model_output_ben.txt', out.values)
Y_bj = np.loadtxt(r"model_output_ben.txt")

In [ ]:
Si_ben = delta.analyze(problem_bj, X_bj, Y_bj, num_resamples=10, conf_level=0.95, print_to_console=True)

## Malawi

In [ ]:
param_values=dfM.iloc[:, : 14]
np.savetxt(r'model_input_mal.txt', param_values.values)
X_mw = np.loadtxt(r"model_input_mal.txt")

In [ ]:
out = dfM["LCOE"]
np.savetxt(r'model_output_mal.txt', out.values)
Y_mw = np.loadtxt(r"model_output_mal.txt")

In [ ]:
Si_mal = delta.analyze(problem_mw, X_mw, Y_mw, num_resamples=10, conf_level=0.95, print_to_console=True)

## Namibia

In [ ]:
param_values=dfN.iloc[:, : 14]
np.savetxt(r'model_input_nam.txt', param_values.values)
X_na = np.loadtxt(r"model_input_nam.txt")

In [ ]:
out = dfN["LCOE"]
np.savetxt(r'model_output_nam.txt', out.values)
Y_na = np.loadtxt(r"model_output_nam.txt")

In [ ]:
Si_nam = delta.analyze(problem_na, X_na, Y_na, num_resamples=10, conf_level=0.95, print_to_console=True)